In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FinalBalancedDataset.csv")

Mounted at /content/drive


In [ ]:
data = data.drop("Unnamed: 0", axis=1)

In [ ]:
data.head()

,Toxicity,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
import re
def prepare_text(text):
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    text = text.split()
    text = ' '.join(text)
    text = word_tokenize(text)
    text = pos_tag(text)
    lemma = []
    for i in text: lemma.append(wordnet_lemmatizer.lemmatize(i[0], pos = get_wordnet_pos(i[1])))
    lemma = ' '.join(lemma)
    return lemma

In [ ]:
data['tweet'] = data['tweet'].apply(lambda x: prepare_text(x))

In [ ]:
data.head(100)

,Toxicity,tweet
0,0,user when a father be dysfunctional and be so ...
1,0,user user thanks for lyft credit i ca n't use ...
2,0,bihday your majesty
3,0,model i love u take with u all the time in ur
4,0,factsguide society now motivation
...,...,...
95,0,user i 'll always hope that one day i 'll get ...
96,0,model i love u take with u all the time in ur
97,0,couple have sex fat naked japanese girl
98,0,hump on that hump day humpers edwardsville pen...


In [ ]:
x=data['tweet']
y=data['Toxicity']
vectorizer=CountVectorizer()
x_vec =vectorizer.fit_transform(x)


In [ ]:
LRmodel = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
LRmodel.fit(x_vec, y)

LogisticRegression(C=2, max_iter=1000, n_jobs=-1)

In [ ]:
d= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Twitter_Data_updated_no_neutral.csv")

In [ ]:
d.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,1.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [ ]:
d=d.dropna(subset=['clean_text','category'])
d['clean_text'] = d['clean_text'].apply(lambda x: prepare_text(x))
d.head()

,clean_text,category
0,when modi promise minimum government maximum g...,-1.0
1,talk all the nonsense and continue all the dra...,1.0
2,what do just say vote for modi welcome bjp tel...,1.0
3,ask his supporter prefix chowkidar their name ...,1.0
4,answer who among these the most powerful world...,1.0


In [ ]:
x= d['clean_text']

x.head()

0    when modi promise minimum government maximum g...
1    talk all the nonsense and continue all the dra...
2    what do just say vote for modi welcome bjp tel...
3    ask his supporter prefix chowkidar their name ...
4    answer who among these the most powerful world...
Name: clean_text, dtype: object

In [ ]:
x_vec = vectorizer.transform(x)

In [ ]:
y_p= LRmodel.predict(x_vec)


In [ ]:
print(y_p)

[1 0 0 ... 0 0 0]


In [ ]:
d['toxicity'] = y_p
d['category'] = d['category'].apply(lambda x: int(x) )
d.head(100)

,clean_text,category,toxicity
0,when modi promise minimum government maximum g...,-1,1
1,talk all the nonsense and continue all the dra...,1,0
2,what do just say vote for modi welcome bjp tel...,1,0
3,ask his supporter prefix chowkidar their name ...,1,0
4,answer who among these the most powerful world...,1,0
...,...,...,...
95,country prospers when the woman the country be...,1,1
96,sabbash mera vote for peppermit abvp,1,0
97,yogi adityanath hold rally seek vote for naren...,1,0
98,from the very beginningmodi do wada faramoshi ...,-1,0
